In [29]:
import locationtagger
import nltk
from pymongo import MongoClient
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium import webdriver
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import os
# from collections import Counter
from dotenv import load_dotenv
load_dotenv()


# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package

True

In [31]:

def getTweetData(tweet):
    try:
        tweetTime = tweet.find_element(
            By.XPATH, ".//time").get_attribute('datetime')
    except NoSuchElementException:
        return
    try:
        # This only gets one image, what if there are more?
        tweetImage = tweet.find_element(
            By.XPATH, ".//div[1]/div[1]//div[2]/div[2]//img").get_attribute("src")
    except NoSuchElementException:
        # print("no image")
        tweetImage = "No Image"
    tweetText = tweet.find_element(
        By.XPATH, ".//div[1]/div[1]/div[2]/div[2]/div[2]").text

    tweetInfo = [tweetText, tweetTime, tweetImage]
    return tweetInfo


def ScraperMain():
    twitterUsername = os.getenv('TW_USERNAME')
    twitterPassword = os.getenv('TW_PASSWORD')

    # Setting up driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://www.twitter.com/login')
    sleep(3)
    # Finding and inputing username
    username = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[5]/label[1]/div[1]/div[2]/div[1]/input[1]")
    username.send_keys(twitterUsername)
    username.send_keys(Keys.RETURN)
    sleep(1)
    # Finding and inputing Password
    # mypassword = getpass()
    password = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/label[1]/div[1]/div[2]/div[1]/input[1]")
    password.send_keys(twitterPassword)
    password.send_keys(Keys.RETURN)
    sleep(5)

    # Selecting and searching required police section
    search = driver.find_element(
        By.XPATH, "//body/div[@id='react-root']/div[1]/div[1]/div[2]/main[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/form[1]/div[1]/div[1]/div[1]/div[1]/label[1]/div[2]/div[1]/input[1]")
    search.send_keys('@TPSOperations')
    search.send_keys(Keys.RETURN)
    sleep(2)

    # Click required page
    SearchQuery = driver.find_element(
        By.XPATH, "//span[contains(text(),'Toronto Police Operations')]")
    SearchQuery.click()

    # Storing Tweet data in list
    tweetData = []
    tweetIds = set()
    lastPos = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    maxDate = False

    while (scrolling and not (maxDate)):
        # Find Tweets
        TPOPageTweets = driver.find_elements(
            By.XPATH, '//article[@data-testid="tweet"]')
        sleep(1)
        for tweet in TPOPageTweets[-100:]:
            currentTweetInfo = getTweetData(tweet)
            if currentTweetInfo:
                tweetId = "".join(currentTweetInfo)
                if tweet not in tweetIds and currentTweetInfo[0] != '' and ":" in currentTweetInfo[0] and 'Good night' not in currentTweetInfo[0] and 'Good afternoon' not in currentTweetInfo[0] and 'Good morning' not in currentTweetInfo[0] and 'Good Night' not in currentTweetInfo[0]:
                    tweetIds.add(tweet)
                    currentTweetInfo.append(tweetId)
                    tweetData.append(currentTweetInfo)
                    if "2023-05-30" in currentTweetInfo[1]:
                        maxDate = True
                        break

        scrollAttempt = 0
        while True:
            driver.execute_script('window.scrollBy(0,3000);')
            sleep(2)
            currPos = driver.execute_script("return window.pageYOffset;")
            if maxDate:
                break
            if lastPos == currPos:
                scrollAttempt += 1
                if scrollAttempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2)
            else:
                lastPos = currPos
                break
    print("Tweets Scraped")
    return tweetData

In [ ]:
# Storing Tweet data in list
tweetData = []
tweetIds = set()
lastPos = driver.execute_script("return window.pageYOffset;")
scrolling = True
maxDate = False

while (scrolling and not (maxDate)):
    # Find Tweets
    TPOPageTweets = driver.find_elements(
        By.XPATH, '//article[@data-testid="tweet"]')
    sleep(1)
    for tweet in TPOPageTweets[-100:]:
        currentTweetInfo = getTweetData(tweet)
        if currentTweetInfo:
            tweetId = "".join(currentTweetInfo)
            if tweet not in tweetIds:
                tweetIds.add(tweet)
                tweetData.append([currentTweetInfo, tweetId])
                # print(currentTweetInfo[1])
                if "2023-04-01" in currentTweetInfo[1]:
                    maxDate = True
                    break

    scrollAttempt = 0
    while True:
        driver.execute_script('window.scrollBy(0,3000);')
        sleep(2)
        currPos = driver.execute_script("return window.pageYOffset;")
        if maxDate:
            break
        if lastPos == currPos:
            scrollAttempt += 1
            if scrollAttempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            lastPos = currPos
            break

In [45]:
tweetData

[['FIRE:\nSinnott Rd & Comstock Rd\n- reports of a fire at a recycling plant\n- police responding\n- \n@Toronto_Fire\n o/s - advised it is a working fire\n- no reported injuries\n- road closures in the area\n- expect delays\n- consider alternate routes\n#GO1038250\n^al',
  '2023-05-09T22:06:30.000Z',
  'No Image',
  'FIRE:\nSinnott Rd & Comstock Rd\n- reports of a fire at a recycling plant\n- police responding\n- \n@Toronto_Fire\n o/s - advised it is a working fire\n- no reported injuries\n- road closures in the area\n- expect delays\n- consider alternate routes\n#GO1038250\n^al2023-05-09T22:06:30.000ZNo Image'],
 ['MISSING:\nHwang, 82\n- last seen on May 8, at 10 a.m., in the Bloor St W & Symington Ave area\n- described as 5\'8", 132 lbs., short grey/white hair\n- last seen wearing black pants, black shoes, black jacket, blue/grey long-sleeve shirt, black hat\n#GO1036302\n^al',
  '2023-05-09T22:00:04.000Z',
  'https://pbs.twimg.com/media/Fvt-lNBXoAUvzKh?format=jpg&name=240x240',
  'MI

In [95]:
def sortData(scrapedTweet):
    # List: catagory (Missing...), Status(Update, Located)
    computedData = []

    for i in range(len(scrapedTweet)):
        dataObject = {}
        if (scrapedTweet[i][0].find('\n') != -1):
            # Getting Status:
            dataObject["Status"] = scrapedTweet[i][0].splitlines()[0].split(":")[0]
            # Getting Updates (?LOCATED)...
            try:
                dataObject["Updates"] = scrapedTweet[i][0].splitlines()[0].split(":")[1]
            except IndexError:
                dataObject["Updates"] = ""
            # currentLocation could be the currentIdentity
            currentLocation = scrapedTweet[i][0].splitlines()[1].split(", ")
            # Checking whether we have a Location or an identity
            if any(char.isdigit() for char in currentLocation):
                dataObject["Name"] = currentLocation[0]
                if currentLocation[1].isnumeric():
                    dataObject["Age"] = currentLocation[1]
                else:
                    dataObject["Age"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:2]
                #Extracting Location from Description
                lists = scrapedTweet[i][0].splitlines()
                del lists[0:2]
                try:
                    partOfDescription = lists[0] + " " + lists[1]
                except IndexError:
                    partOfDescription = scrapedTweet[i][0]                   
                dataObject["Location"] = locationtagger.find_locations(text = partOfDescription).other
            else:
                try:
                    dataObject["Location"] = scrapedTweet[i][0].splitlines()[1]
                except IndexError:
                    dataObject["Location"] = ""
                description = scrapedTweet[i][0].splitlines()
                del description[0:1]
            if dataObject["Location"] is not None:
                locationcheck = dataObject["Location"]
                print(locationcheck)
                print(bool(dataObject["Location"]))
                if type(dataObject["Location"]) == list and bool(dataObject["Location"]):
                    res = max(locationcheck, key = len)
                    locationcheck = res
                if "and" in locationcheck:
                    locationcheck = locationcheck.replace("and", "")
                if "&" in locationcheck:
                    locationcheck = locationcheck.replace("&", "")
                geolocator = Nominatim(user_agent="Twitter_Scraper")
                geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
                geocode2 = RateLimiter(geocode, min_delay_seconds=1)
                locationcor = geocode2(locationcheck)
                if locationcor is not None:
                    locationcor2 = [locationcor.latitude, locationcor.longitude]
                    dataObject["LocationGoeCode"] = locationcor2
                else:
                    dataObject["LocationGoeCode"] = ["Coordinates not found"]
                    
            else:
                dataObject["LocationGoeCode"] = ["No Location Provided"]
            dataObject["TweetedTime"] = scrapedTweet[i][1]
            dataObject["ImageUrl"] = scrapedTweet[i][2]
            dataObject["Description"] = description
            computedData.append(dataObject)
        else:
            print("Tweet Skipped: ", scrapedTweet[i])
    print("Tweets Sorted")
    return(computedData)
        


In [5]:

def addToDB(computedData):
    # Get the database using the method we defined in pymongo_test_insert file
    # from pymongo_get_database import get_database
    MONGODB_PASS = os.getenv('MONGODB_PASS')

    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb+srv://OCMB1:" + MONGODB_PASS + \
        "@cluster0.e5wloiz.mongodb.net/?retryWrites=true&w=majority"

    # Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
    client = MongoClient(CONNECTION_STRING)

    # print(client.drop_database('Toronto_Police_Crime_Report'))
    # Create the database for our example (we will use the same database throughout the tutorial
    if 'Toronto_Police_Crime_Report' in client.list_database_names():
        client.drop_database('Toronto_Police_Crime_Report')

    db = client['Toronto_Police_Crime_Report']
    collection_name = db["tweetsData"]
    res = collection_name.insert_many(computedData)
    print("Added To DataBase")

In [ ]:
tweets = ScraperMain()
sortedData = sortData(tweets)
addToDB(sortedData)

In [69]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Twitter_Scraper")
geocode = lambda query: geolocator.geocode("%s, Toronto ON" % query)
location = geocode("Finch Ave W  HWY 27 N")
location2 = [location.latitude, location.longitude]
location2
# print((location.latitude, location.longitude))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [96]:
sortedData = sortData(tweets)

Keele St and Sheppard Ave
True
Keele St and Sheppard Ave
True
['June', 'Gwynne Ave', 'Queen St']
True
Finch Ave W & HWY 27 N
True
Strachan Ave & East Liberty St
True
['June', 'the Avenue Rd', 'Eglinton', 'Avenue Rd', 'Eglinton Ave']
True
['Joyce']
True
Sutharsan, 30 
True
['June', 'Ontario - Derick resides', 'Derick', 'Finch Ave', 'Willowdale Ave']
True
['Derick']
True
['June', 'the Sherbourne St', 'Wellesley St E', 'Sherbourne St', 'Wellesley St']
True
['June', 'the York Mills Rd & Victoria Park Ave', 'York Mills Rd', 'Victoria Park Ave']
True
['June', 'the Sherbourne St', 'Wellesley St E', 'Sherbourne St', 'Wellesley St']
True
['June', 'the York Mills Rd & Victoria Park Ave', 'York Mills Rd', 'Victoria Park Ave']
True
['Matthew']
True
Keele St & Doverhouse Ave
True
['the Sherbourne St & Gerrard St E area', 'June', 'Sherbourne St', 'Gerrard St']
True
['June', 'Steeles Ave', 'W & Islington Ave', 'Steeles']
True
['Winnifred']
True
Gardiner Expressway W & Spadina Ave
True
Gardiner Expres

In [98]:
sortedData

[{'Status': 'COLLISION',
  'Updates': '',
  'Location': 'Keele St and Sheppard Ave',
  'LocationGoeCode': [43.7450075, -79.4860809],
  'TweetedTime': '2023-06-08T22:51:38.000Z',
  'ImageUrl': 'No Image',
  'Description': ['Keele St and Sheppard Ave',
   '6:43 pm',
   '-reports of a 2-vehicle collision',
   '-police o/s investigating',
   '- ',
   '@toronto_fire',
   ' o/s',
   '-unknown injuries',
   '-expect delays in the area',
   '-consider alternate routes',
   '#GO1312364',
   '^lm']},
 {'Status': 'COLLISION',
  'Updates': ' (UPDATE)',
  'Location': 'Keele St and Sheppard Ave',
  'LocationGoeCode': [43.7450075, -79.4860809],
  'TweetedTime': '2023-06-08T23:33:34.000Z',
  'ImageUrl': 'No Image',
  'Description': ['Keele St and Sheppard Ave',
   '6:43 pm',
   '-one vehicle into pole',
   '-fire and police still o/s',
   '-no injuries',
   '-delays in the area, consider alternate routes',
   '#GO1312364',
   '^lm']},
 {'Status': 'MISSING',
  'Updates': '',
  'Name': 'Shayana',
  'Age